In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
np.set_printoptions(precision=5, suppress=True, linewidth=100)
plt.rcParams['figure.dpi'] = 150
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import tebd
from tenpy.networks.mps import MPS
from tenpy.models.model import CouplingMPOModel
from tenpy.networks import site
from tenpy.networks.site import Site
import matplotlib.pyplot as plt
tenpy.tools.misc.setup_logging(to_stdout="INFO")
import time
import csv
import pandas as pd

## 3 level site for the Potts model with Z3 charge conservation

In [3]:
class PottsSite(Site):

    def __init__(self, conserve='Z3'):
        if not conserve:
            conserve = 'None'
        if conserve not in ['None','Z3']:
            raise ValueError("invalid `conserve`: " + repr(conserve))

        
        qcharges = np.array([0,1,2])
        # sigma and tau operators for the definition of the Potts model
        tau = np.array([[1,0,0],[0,np.exp(1j*2*np.pi/3),0],[0,0,np.exp(-1j*2*np.pi/3)]])
        taudag = np.array([[1,0,0],[0,np.exp(-1j*2*np.pi/3),0],[0,0,np.exp(1j*2*np.pi/3)]])
        sigma = np.array([[0,1,0],[0,0,1],[1,0,0]])
        sigmadag = np.array([[0,0,1],[1,0,0],[0,1,0]])
        
        # Z3 charge operator and its powers
        Q=1j*(taudag-tau)/np.sqrt(3)
        Q2=Q@Q
        Q4=Q2@Q2
        ops = dict(sigma=sigma, sigmadag=sigmadag,tau=tau,taudag=taudag,Q=Q,Q2=Q2)
        if conserve == 'Z3':
            chinfo = npc.ChargeInfo([3], ['Z3_potts'])
            leg = npc.LegCharge.from_qflat(chinfo, np.array(qcharges, dtype=np.int64))
        else:
            leg = npc.LegCharge.from_trivial(3)
        self.conserve = conserve
        names = [str(i) for i in range(3)]
        Site.__init__(self, leg, names, **ops)

## Single copy 3-state Potts model with S3 symmetry for benchmarking and testing

In [4]:
class Potts(CouplingMPOModel):
    default_lattice = "Chain"
    force_default_lattice=True
    def init_sites(sefl,model_params):
        spin = PottsSite(conserve= "Z3")
        return spin
    def init_terms(self, model_params):
        J= model_params.get("J",1.)
        U = model_params.get("U", 1.)
        self.add_onsite(-J,0,"tau",plus_hc=True)
        self.add_coupling(-U,0,"sigma",0,"sigmadag",[1],plus_hc=True)

## DMRG for groundstate search  in the single copy Potts model

In [ ]:
#system sizes
Ls = [8]

# coupling constants
Js = np.linspace(0,2,10)

# correlation functions with sigma operator
cors_sigma=np.zeros((len(Ls),len(Js)))

# correlation functions with the Z3 charge operator
cors_charge=np.zeros((len(Ls),len(Js)))

for i,L in enumerate(Ls):
    for j,J in enumerate(Js):
        M= Potts({"L": L, "U": 1, "J":J})
        product_state = ('0')*(L)
        #psi = MPS.from_lat_product_state(M.lat, [['up']])
        psi0=MPS.from_product_state(M.lat.mps_sites(),product_state)
        eng = tenpy.algorithms.dmrg.SingleSiteDMRGEngine(psi0, M, {'mixer': True,
                                                                            'trunc_params': {'chi_max': 300, 'svd_min':1e-5}, 'max_sweeps':24,'max_E_err':1.0e-04})
        E,psi=eng.run()
        cors_sigma[i,j]=float(psi.expectation_value_term([("sigmadag",int(L/4)),("sigma",int(3/4*L))]))
        cors_charge[i,j]=float(psi.expectation_value_term([("Q",int(L/4)),("Q",int(3/4*L))]))
        

## Definition of the 2-copy Potts model (Brownian circuit in the doubled Hilbert space)

In [5]:
class Potts_2copy(CouplingMPOModel):
    """Average time evolution in doubled Hilbert space of random TFI"""

    default_lattice = "Chain"
    force_default_lattice = True

    def init_sites(self, model_params):
        spin_f = PottsSite(conserve= "Z3")
        spin_b = PottsSite(conserve = "Z3")
        sites = [spin_f, spin_b]
        
        site.set_common_charges(sites, new_charges='independent')
        return [spin_f, spin_b], ['sf', 'sb']  # return species names as well!
        # this causes `init_lattice` to initialize a `MultiSpeciesLattice`
        
    def init_terms(self, model_params):
        J = model_params.get('J', 1.)
        U = model_params.get('U', 1.)
        s = model_params.get('s', 0.5)
        p = model_params.get('p', 0.1)


       # self.add_onsite(0.0001,0,'sigma',plus_hc=True)
        self.add_onsite(U,0,'tau',plus_hc=True)
        self.add_onsite(U,1,'tau',plus_hc=True)

        self.add_coupling(J, 0, 'sigma', 0, 'sigmadag', [1],plus_hc=False)
        self.add_coupling(J, 1, 'sigma', 1, 'sigmadag', [1],plus_hc=False)
        self.add_coupling(J, 0, 'sigmadag', 0, 'sigma', [1],plus_hc=False)
        self.add_coupling(J, 1, 'sigmadag', 1, 'sigma', [1],plus_hc=False)

        self.add_coupling(-2*U,0,'tau',1,'taudag',[0])
        self.add_coupling(-2*U,0,'taudag',1,'tau',[0])
        self.add_coupling(-2*U,0,'tau',1,'tau',[0])
        self.add_coupling(-2*U,0,'taudag',1,'taudag',[0])
        
        self.add_multi_coupling(-2*J,[('sigmadag',[0],0),('sigma',[1],0),('sigmadag',[0],1),('sigma',[1],1)])
        self.add_multi_coupling(-2*J,[('sigmadag',[0],0),('sigma',[1],0),('sigma',[0],1),('sigmadag',[1],1)])
        self.add_multi_coupling(-2*J,[('sigma',[0],0),('sigmadag',[1],0),('sigmadag',[0],1),('sigma',[1],1)])
        self.add_multi_coupling(-2*J,[('sigma',[0],0),('sigmadag',[1],0),('sigma',[0],1),('sigmadag',[1],1)])

        #postselection terms
        self.add_onsite(p,0,'Q2',plus_hc=False)
        self.add_onsite(p,1,'Q2',plus_hc=False)

        self.add_coupling(-p*(2-s),0,'Q2',1,'Q2',[0],plus_hc=False)
        
        
        

## DMRG for groundstate search of 2-copy model

In [ ]:
# array of system sizes
Ls= [8]

#array for measurement rates
ps=np.linspace(0.01,12,12)


#sigma-sigma correlations
cors = np.zeros((len(Ls),len(ps)),dtype = np.complex64)

#charge-charge correlations
cors2 = np.zeros((len(Ls),len(ps)),dtype = np.complex64)

for i,L in enumerate(Ls):
    for j,p in enumerate(ps):
        M=Potts_2copy({'L':L,'p':p,'U':1,'J':1,'s':0.5})
        product_state = ('0','0')*(L)
        psi0=MPS.from_product_state(M.lat.mps_sites(),product_state)
        eng = tenpy.algorithms.dmrg.SingleSiteDMRGEngine(psi0, M, {'mixer_params': {"amplitude": 1e-5},
                                                                   'trunc_params': {'chi_max': 400, 'svd_min':1e-4}, 'max_sweeps':30,'max_E_err':1.0e-12})
        E,psi=eng.run()
        psi0=MPS.from_product_state(M.lat.mps_sites(),product_state)
        print(psi.overlap(psi0))
        print("-------------------------------------------------------------------------------")
        print()
        c=complex(psi.expectation_value_term([('sigmadag',int((L/4-1)*2)),('sigmadag',int((L/4-1)*2)+1),('sigma',int(L-L/4-1)*2),('sigma',int(L-L/4-1)*2+1)]))
        print(c)
        c2=complex(psi.expectation_value_term([('Q',int((L/4-1)*2)),('Q',int((L/4-1)*2)+1),('Q',int(L-L/4-1)*2),('Q',int(L-L/4-1)*2+1)]))
        print(c2)
        print("-------------------------------------------------------------------------------")
        cors[i,j]=c
        cors2[i,j]=c2
        